# Исследование надёжности заёмщиков
## Описание проекта
Нужно научиться определять рыночную стоимость объектов недвижимости по данным проекта Яндекс Недвижимость. Задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

# 1. Обзор данных

Изучим данные, предоставленные для исследования. Импортируем библиотеку *pandas* и сохраним таблицу в переменную *df*.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/real_estate_data.csv')

Заглянем одним глазком в первые 5 строк таблицы.

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Теперь узнаем общую информацию о таблице.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов разных типов (числовых и строковых). В двух столбцах имеются пропущенные значения.

Не всегда по названию колонки можно определить, какие данные она в себе содержит. Поэтому, обратимся к документации к данным для получения информации о столбцах:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

**Выводы**

Каждая строка таблицы содержит информацию о клиенте банка - потенциальном получателе кредита. Для решения задач данного исследования наиболее ценными являются столбцы *children*, *family_status* и *debt*. В столбцах *days_employed* и *total_income* имеются пропущенные значения. Эту и другие проблемы нам нужно будет решить на следующем шаге проекта - этапе предобработки данных.

# 2. Предобработка данных

Узнаем, сколько пропусков в таблице, вызовом набора методов для суммирования пропущенных значений.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверим, какую долю составляют пропуски в каждом из столбцов *days_employed* и *total_income*.

In [6]:
df.isna().sum() / df.count()

children            0.000000
days_employed       0.112346
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.112346
purpose             0.000000
dtype: float64

Чуть больше 11 %. Не хило!
Количество пропущенных значений в столбце с доходами соответсвует количеству пропусков в колонке с трудовым стажем. Возможно между этими пропусками есть связь. Проверим эту гипотезу.

In [7]:
#выведем сумму пропусков в столбце total_income в строках с пропущенными значениями в days_employed
df[df['days_employed'].isna()]['total_income'].isna().sum()

2174

Гипотеза подтвердилась! Клиенты, не указавшие свой доход, так же не предоставили информацию и о своём стаже. Возможно они все бедные студенты? 

Выделим возраст и тип занятости клиентов в строках с пропусками. 

In [8]:
# подсчитаем количество уникальных значений в столбцах 'dob_years' и 'income_type' методом value_counts()
print(df[df['days_employed'].isna()]['dob_years'].value_counts().head(10))
df[df['days_employed'].isna()]['income_type'].value_counts().head(10)

34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
Name: dob_years, dtype: int64


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Преобладают сотрудники компаний среднестатистического возраста. Всё же, видимо, пропуски связаны с технической ошибкой при записи или выгрузке данных. 

Для дальнейшего анализа нам потребуется избавиться от пропусков в таблице. Подумаем, каким образом лучше это сделать. 

Проверим отличия максимального, минимального и среднего значений дохода клиентов.

In [9]:
print(f'Минимальный доход: {df["total_income"].min():0.2f}')
print(f'Средний доход: {df["total_income"].mean():0.2f}')
print(f'Максимальный доход: {df["total_income"].max():0.2f}')

Минимальный доход: 20667.26
Средний доход: 167422.30
Максимальный доход: 2265604.03


Как мы видим, в данных с доходами клиентов есть выдающиеся значения, которые в несколько раз превосходят низкие показатели. В таком случае, средняя температура по больнице не отражает реальное положение дел с финансовым положением кредитополучателей. 

Поэтому, заполним пропуски в этом столбце медианным значением —  это лучшее решение для количественных переменных.

In [10]:
# применим метод fillna() для заполнения пропусков, заменим тип значений столбца на int и проверим внесение изменений
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


Настала очередь заняться столбцом с трудовым стажем.
Выведем аналогично максимальные и минимальные значения.

In [11]:
print(f'Минимальный стаж: {df["days_employed"].min():0.2f}')
print(f'Максимальный стаж: {df["days_employed"].max():0.2f}')

Минимальный стаж: -18388.95
Максимальный стаж: 401755.40


 Либо среди клиентов есть Бенджамины Баттоны, либо мы нашли аномальные данные. Стаж не может быть отрицательным. Исправим это дело.

In [12]:
# выясним долю отрицательных значений от всех значений
df[df['days_employed'] < 0]['days_employed'].count() / df['days_employed'].count()

0.8219730246498889

82%! Почти все значения столбца с трудовым стажем ниже нуля. Возможно, на это повлияли какие-то неполадки с системой отсчета, либо коварный минус затесался на этапе выгрузки данных из базы. В таком случае безболезненно удалить ошибочные данные из выборки не получится. 

Учитывая, что данные стажа не влияют на задачи текущего исследования, не будем тратить время на выяснение причин ошибки, и заменим все отрицательные значения их модулями.

In [13]:
# избавимся от отрицательных значений и выведем минимальное значение стажа для проверки
df["days_employed"] = df["days_employed"].abs()
print(f'Минимальный стаж: {df["days_employed"].min():0.2f}')

Минимальный стаж: 24.14


Готово. Теперь можем заменить пропуски в столбце медианой.

In [14]:
# применим метод fillna() для заполнения пропусков и проверим внесение изменений
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Теперь проверим и другие столбцы на наличие артефактов.

In [15]:
# подсчитаем количество уникальных значений в столбцах 'children' и 'debt'
df['debt'].value_counts()
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

У 76 клиентов 20(!) детей. А ещё у нескольких - их отрицательное количество. Доля этих значений менее 1 %, их удаление не повлияет на результаты анализа, можем избавиться от строк с аномальными значениями.

In [16]:
# удалим строки с аномальными значениями и выполним проверку удаления
df= df[(df['children'] != 20) & (df['children'] >= 0)]
df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Далее, необходимо установить наличие дубликатов.  Если найдутся, удаляем, и проверяем, все ли удалились.

In [17]:
# получение суммарного количества дубликатов в таблице df
df.duplicated().sum()

54

In [18]:
# удаление всех дубликатов из таблицы df специальным методом и сброс индексов
df = df.drop_duplicates().reset_index(drop=True)

In [19]:
# проверка на отсутствие
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами их появления в базе данных.

Далее выполним поиск неявных дубликатов в таблице. Ещё в самом начале исследования, по первым строкам таблицы мы заметили, что в столбце *education* одни и те же значения записаны по-разному: с использованием заглавных и строчных букв.

In [20]:
# приведем все значения столбца к нижнему регистру с помощью метода lower()
df['education'] = df['education'].str.lower()
# проверим изменения через вывод уникальных значений столбца
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

С этим столбцом разобрались. Пройдемся теперь по другим столбцам, содержащим категорийные значения.

In [21]:
# вывод уникальных значений в столбцах
df['family_status'].unique()
df['gender'].unique()
df['income_type'].unique()
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из всех подвергнутых проверке серий неявные дубликаты содержатся только в столбце *purpose*. Для их обработки можно применить лемматизацию.

Создадим функцию, которая на основании данных из столбца *purpose* сформирует новый столбец *purpose_category*, в который войдут следующие категории:
- операции с автомобилем;
- операции с недвижимостью;
- проведение свадьбы;
- получение образования.

In [22]:
# напишем функцию категоризации
def purpose_ctgry(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образов' in purpose:
        return 'получение образования'

# создадим новый столбец с помощью метода apply()
df['purpose_category'] = df['purpose'].apply(purpose_ctgry)

# проверим, все ли причины нашли свою категорию
df['purpose_category'].isna().sum()

0

Переходя далее, можем заметить, что столбцы:
- *education_id* и *education*;
- *family_status_id* и *family_status*

по сути являются избыточными и дублируют друг друга. Перенесём их в отдельные таблицы-"справочники".

In [23]:
# в датафрейм "family_status" перенесем соответсвующие столбцы, оставив только уникальные соответствия
family_status = df.loc[:, ['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
display(family_status.head(10))

# в датафрейм "education" перенесем соответсвующие столбцы, оставив только уникальные соответствия
education = df.loc[:, ['education_id', 'education']].drop_duplicates().reset_index(drop=True)
education.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [24]:
# удалим лишние столбцы из исходного датафрейма с помощью метода drop()
df = df.drop(columns=['education', 'family_status'], axis=1)
df.columns           

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'purpose_category'],
      dtype='object')

На основании диапазонов, указанных ниже, создадим столбец *total_income_category* с категориями доходов:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [25]:
# напишем функцию категоризации
def income_category(total_income):
    if total_income <= 30000:
        return 'E'
    elif 30000 < total_income <= 50000:
        return 'D'
    elif 50000 < total_income <= 200000:
        return 'C'
    elif 200000 < total_income <= 1000000:
        return 'B'
    else:
        return 'A'

# создадим новый столбец с помощью метода apply()
df['total_income_category'] = df['total_income'].apply(income_category)
df.head()        

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C


**Выводы**

На этапе предобработки в данных обнаружились пропуски, избыточные столбцы, аномальные значения и всяческие виды дубликатов. Исправление этих ошибок позволит провести анализ точнее. Обширные значения доходов клиентов и целей получения кредитов были объединены в удобные для анализа категории. 

# 3. Ход исследования

## Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [26]:
# сгруппируем клиентов по количеству детей и сравним количество должников и отношение должников ко всем клиентам из категории
debt_by_children = df.groupby('children').agg({'debt': ['count', 'sum', 'mean']})
debt_by_children

debt                
          count   sum      mean
children                       
0         14107  1063  0.075353
1          4809   444  0.092327
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

**Выводы**

Бездетные клиенты чаще погашают задолженности вовремя, чем клиенты с детьми. Оно и понятно, вы цены на детские коляски видели? При этом, само количество детей (когда их больше 1) слабо коррелирует с фактом возврата кредита в срок. 

## Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
# сгруппируем клиентов по семейному положению и сравним количество должников и отношение должников ко всем клиентам из категории
debt_by_family = df.groupby('family_status_id').agg({'debt': ['count', 'sum', 'mean']})
debt_by_family.set_axis(['total_client', 'debt_client', 'mean_debt'], axis='columns', inplace=True) 
# достанем категории из справочника, объединив таблицы с помощью метода merge()
res = family_status.merge(debt_by_family, on='family_status_id', how='right')
res

,family_status_id,family_status,total_client,debt_client,mean_debt
0,0,женат / замужем,12266,927,0.075575
1,1,гражданский брак,4146,385,0.092861
2,2,вдовец / вдова,951,63,0.066246
3,3,в разводе,1189,84,0.070648
4,4,Не женат / не замужем,2796,273,0.097639


**Выводы**

Удивительно, но факт: холостые и незамужние клиенты - главные должники. Сюда же можно отнести и клиентов с неузаконенным семейным статусом - тех, кто живёт в гражданском браке. В среднем женатые/замужние клиенты имеют на 2 % меньше долгов.  

## Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
# сгруппируем клиентов по уровню дохода и сравним количество должников и отношение должников ко всем клиентам из категории
debt_by_income = df.groupby('total_income_category').agg({'debt': ['count', 'sum', 'mean']})
debt_by_income

debt                
                       count   sum      mean
total_income_category                       
A                         25     2  0.080000
B                       5013   354  0.070616
C                      15939  1353  0.084886
D                        349    21  0.060172
E                         22     2  0.090909

**Выводы**

Для анализа должников из категории с самыми высокими и с самыми низкими доходами недостаточно данных - в нашей огромной выборке всего 47 человек попадают в эти категории, и среди них лишь 4 должника. По остальным категориям факт возврата кредита никак не коррелирует с уровнем дохода. Это контринтуитивно, но цифры не врут.

## Как разные цели кредита влияют на его возврат в срок?

In [29]:
# сгруппируем клиентов по уровню дохода и сравним количество должников и отношение должников ко всем клиентам из категории
debt_by_purpose = df.groupby('purpose_category').agg({'debt': ['count', 'sum', 'mean']})
debt_by_purpose

debt               
                          count  sum      mean
purpose_category                              
операции с автомобилем     4281  400  0.093436
операции с недвижимостью  10754  780  0.072531
получение образования      3989  369  0.092504
проведение свадьбы         2324  183  0.078744

**Выводы**

Кредиты, полученные для операций с недвижимостью, погашают гораздо чаще других. Возможно, это связано с льготированием ипотечных программ или, скорее всего, с серьезностью подхода к такого рода покупке. В случае невыплаты ипотеки, клиенты могут остаться без жилья, поэтому очень ответственно подходят к своим обязательствам. 

А вот хуже всего дело обстоит с автокредитами и кредитами на обучение. Автомобили покупаются под более высокий процент, чем те же квартиры, и часто предполагают много сопутствующих затрат на обслуживание, ремонт в случае аварий и др., возможно клиенты не всегда учитывают эти риски в полном объеме, и в этом кроется основа этой статистики. 

Кредиты на обучение берутся студентами без стабильного заработка - в этом статусе тяжело совмещать и учёбу, и работу. Во время учебы студенты часто не могут устроиться на полноценную работу ввиду отсутствия временных ресурсов, и в основном перебиваются подработками. Как следствие, прогнозируемо низкий уровень дохода.

# 4. Результаты исследования

Рабочие гипотезы: количество детей, семейное положение, уровень дохода клиента и разные цели кредита коррелируют с фактом погашения кредита в срок.

**Общие результаты**

Бездетные клиенты чаще погашают кредиты в срок, чем клиенты с детьми. При этом количество детей несильно коррелирует с фактом возврата денег. Женатые клиенты  более дисциплинированы, чем холостые. Удивительно, но уровень дохода никак не влияет на факт возврата кредита в срок. От цели кредита зависит вероятность возврата денег обратно. Если цель серьезная - как покупка квартиры - то шанс получить деньги обратно выше, нежели шанс возврата кредита на обучение студентом.